# Full C3D object structure:

# Top Level (c)

## header

### points

- size
- frame_rate
- first_frame
- last_frame

### analogs

- size
- frame_rate
- first_frame
- last_frame

### events

- size
- events_time
- events_label

## parameters

### POINT

- __METADATA__
- USED
- SCALE
- RATE
- DATA_START
- FRAMES
- UNITS
- LABELS
- DESCRIPTIONS

### ANALOG

- __METADATA__
- USED
- LABELS
- GEN_SCALE
- SCALE
- OFFSET
- GAIN
- UNITS
- RATE
- RATIO
- FORMAT
- DESCRIPTIONS
- BITS

### FORCE_PLATFORM

- __METADATA__
- USED
- TYPE
- ZERO
- CORNERS
- ORIGIN
- CHANNEL
- ZEROS
- CAL_MATRIX

### MANUFACTURER

- __METADATA__
- COMPANY
- SOFTWARE
- VERSION_LABEL

### SUBJECTS

- __METADATA__
- USED
- LABEL_PREFIXES
- NAMES

### FORCE_STRUCTURE

- __METADATA__
- USED

### EVENT

- __METADATA__
- USED

## data

### points

numpy array: 4 x number of markers x number of point frames

### meta_points

- residuals
- camera_masks

### analogs

numpy array: 1 x 6*number of force plates x number of analog frames

In [1]:
import ezc3d
import numpy as np
from glob import glob
import matplotlib.pyplot as plt
import pandas as pd
%matplotlib inline
import os
from tqdm import tqdm

- type : OH [overhead], TQ [three quarter], SD [side], UD [under]
- strike : S [strike], B [ball]
- 최종 파일 저장 형태 : 이름_트라이이얼_오른왼쪽_몸무게_키_타입_볼스피드_스트라이크볼

In [2]:
day = '20240302'

In [3]:
kinematic_list = [i.replace('\\','/') for i in sorted(glob(f"c3d file/{day}/kinematic/**/*c3d"))]
kinetic_list = [i.replace('\\','/') for i in sorted(glob(f"c3d file/{day}/kinetic/**/*c3d"))]

kinematic_name = [os.path.basename(i).split('.')[0] for i in kinematic_list]
kinetic_name = [os.path.basename(i).split('.')[0] for i in kinetic_list]

In [4]:
kinematic_name

['songseokhyun_0001',
 'songseokhyun_0002',
 'songseokhyun_0003',
 'songseokhyun_0004',
 'songseokhyun_0005',
 'songseokhyun_0006',
 'songseokhyun_0007',
 'songseokhyun_0008',
 'songseokhyun_0009',
 'songseokhyun_0010',
 'songseokhyun_0011',
 'songseokhyun_0012',
 'songseokhyun_0013',
 'songseokhyun_0014',
 'songseokhyun_0015']

In [5]:
players = pd.read_excel(f'c3d file/{day}/player_information.xlsx',index_col = 'trial')
players.index = kinematic_name
players['weight'] = 91
players['height'] = 185
players

,ball_speed,strike,weight,height,type,Unnamed: 6,Unnamed: 7,Unnamed: 8
songseokhyun_0001,88,B,91,185,OH,NaN,NaN,88.1
songseokhyun_0002,87,B,91,185,OH,NaN,NaN,87.4
songseokhyun_0003,88,B,91,185,OH,NaN,NaN,87.6
songseokhyun_0004,88,B,91,185,OH,NaN,NaN,88.0
songseokhyun_0005,88,B,91,185,OH,NaN,NaN,87.8
songseokhyun_0006,86,B,91,185,OH,NaN,NaN,86.4
songseokhyun_0007,86,B,91,185,OH,NaN,NaN,86.3
songseokhyun_0008,85,B,91,185,OH,NaN,NaN,85.4
songseokhyun_0009,84,B,91,185,OH,NaN,NaN,84.0
songseokhyun_0010,85,B,91,185,OH,NaN,NaN,85.2


In [6]:
kinematic_c3d = {}
for i, name in zip(kinematic_list, kinematic_name):
    if 'right' in i:
        side = 'R'
    elif 'left' in i:
        side = 'L'
    
    weight = str(players.loc[name]['weight'])
    height = str(players.loc[name]['height'])
    pitchtype = players.loc[name]['type']
    ballspeed = str(players.loc[name]['ball_speed'])
    strikeball = players.loc[name]['strike']
    
    kinematic_c3d[f"{name}_{side}_{str(weight)}_{str(height)}_{pitchtype}_{ballspeed}_{strikeball}"] = ezc3d.c3d(i,extract_forceplat_data=True)
    
kinetic_c3d = {}
for i, name in zip(kinetic_list, kinetic_name):
    if 'right' in i:
        side = 'R'
    elif 'left' in i:
        side = 'L'
        
    weight = str(players.loc[name]['weight'])
    height = str(players.loc[name]['height'])
    pitchtype = players.loc[name]['type']
    ballspeed = str(players.loc[name]['ball_speed'])
    strikeball = players.loc[name]['strike']
    kinetic_c3d[f"{name}_{side}_{str(weight)}_{str(height)}_{pitchtype}_{ballspeed}_{strikeball}"] = ezc3d.c3d(i, extract_forceplat_data=True)

### Combine Kinematic C3D File & Force Plate C3D File

In [7]:
wrong = []
for file in tqdm(kinematic_c3d):
    try:
        theia = kinematic_c3d[file]
        device = kinetic_c3d[file]

        theia['header']['analogs']['size'] = device['header']['analogs']['size']
        theia['header']['analogs']['frame_rate'] = device['header']['analogs']['frame_rate']
        theia['header']['analogs']['first_frame'] = device['header']['analogs']['first_frame']
        theia['header']['analogs']['last_frame'] = device['header']['analogs']['last_frame']
        theia['parameters']['FORCE_PLATFORM'] = device['parameters']['FORCE_PLATFORM']
        theia['parameters']['PROCESSING'] = device['parameters']['PROCESSING']
        theia['parameters']['POINT'] = device['parameters']['POINT']
        theia['parameters']['POINT']['UNITS']['value'] = ['mm']
        theia['parameters']["FORCE_PLATFORM"]['CORNERS']['value'] = device['parameters']["FORCE_PLATFORM"]['CORNERS']['value']
        theia["parameters"]["ANALOG"] = device['parameters']['ANALOG']
        theia.add_parameter("ANALOG", "RATIO",description="Analog to Point Ratio", value=6)

        t_frame = theia['data']['points'].shape[-1] # 떼이아 프레임
        d_frame = device['data']['points'].shape[-1] # 아날로크 프레임

        if t_frame == d_frame:
            theia['data']['points'] = device['data']['points']
            theia['data']['analogs'] = device['data']['analogs']
            theia['data']['platform'] = device['data']['platform']
            
        elif t_frame < d_frame:
            theia['data']['points'] = device['data']['points'][:,:,:t_frame]
            theia['data']['analogs'] = device['data']['analogs'][:,:,:int(t_frame * 6)]
            theia['data']['platform'] = device['data']['platform']
            
        elif t_frame > d_frame:
            theia['data']['points'] = device['data']['points'][:,:,:d_frame]
            theia['data']['analogs'] = device['data']['analogs'][:,:,:int(d_frame * 6)]
            theia['data']['rotations'] = theia['data']['rotations'][:,:,:,:d_frame]
            theia['data']['platform'] = device['data']['platform']
            
        theia.write(f'c3d file/{day}/inte/{file}')
        
    except Exception:
        print('오류 :',file)
        wrong.append(file)

100%|██████████| 15/15 [00:00<00:00, 35.79it/s]


In [8]:
theia['parameters'].keys()

dict_keys(['MANUFACTURER', 'THEIA3D', 'PROCESSING', 'TRIAL', 'POINT', 'ROTATION', 'ANALOG', 'EVENT', 'DATA_BLOCKS', 'FORCE_PLATFORM'])

In [9]:
label = theia['parameters']['ROTATION']['LABELS']['value']
rot_mat = theia['data']['rotations']

In [10]:
ROT_MAT = {}
for i in range(rot_mat.shape[2]):
    ROT_MAT[label[i]] = rot_mat[:,:,i,:]

In [11]:
ROT_MAT['head_4X4'].shape

(4, 4, 829)

In [12]:
wrong

[]

In [13]:
# file = 'kjy_0009_R_86_184_OH_143_S'

# theia = kinematic_c3d[file]
# device = kinetic_c3d[file]


# theia['header']['analogs']['size'] = device['header']['analogs']['size']
# theia['header']['analogs']['frame_rate'] = device['header']['analogs']['frame_rate']
# theia['header']['analogs']['first_frame'] = device['header']['analogs']['first_frame']
# theia['header']['analogs']['last_frame'] = device['header']['analogs']['last_frame']
# theia['parameters']['FORCE_PLATFORM'] = device['parameters']['FORCE_PLATFORM']
# theia['parameters']['POINT'] = device['parameters']['POINT']
# theia['parameters']['POINT']['UNITS']['value'] = ['mm']
# theia['parameters']['PROCESSING'] = device['parameters']['PROCESSING']
# theia['parameters']["FORCE_PLATFORM"]['CORNERS']['value'] = device['parameters']["FORCE_PLATFORM"]['CORNERS']['value']
# theia["parameters"]["ANALOG"] = device['parameters']['ANALOG']
# theia.add_parameter("ANALOG", "RATIO",description="Analog to Point Ratio", value=6)

# t_frame = theia['data']['points'].shape[-1] # 떼이아 프레임
# d_frame = device['data']['points'].shape[-1] # 아날로크 프레임

# if t_frame == d_frame:
#     print('1', file)
#     theia['data']['points'] = device['data']['points']
#     theia['data']['analogs'] = device['data']['analogs']
#     theia['data']['platform'] = device['data']['platform']
    
# elif t_frame < d_frame:
#     print('2', file)
#     theia['data']['points'] = device['data']['points'][:,:,:t_frame]
#     theia['data']['analogs'] = device['data']['analogs'][:,:,:int(t_frame * 6)]
#     theia['data']['platform'] = device['data']['platform']
    
# elif t_frame > d_frame:
#     print('3', file)
#     theia['parameters']['TRIAL']['ACTUAL_END_FIELD']['value'] = np.array(([[d_frame],
#                                                                     [  0]]))
#     theia.add_parameter("ROTATION", "RATIO", 1)
#     theia['data']['points'] = device['data']['points'][:,:,:d_frame]
#     theia['data']['analogs'] = device['data']['analogs'][:,:,:int(d_frame * 6)]
#     theia['data']['rotations'] = theia['data']['rotations'][:,:,:,:d_frame]
#     theia['data']['platform'] = device['data']['platform']
    
# theia.write(f'c3d file/{day}/inte/{file}')


| c3d_f = c3d(file_list[0], extract_forceplat_data=True)             |
|--------------------------------------------------------------------|
|--------------------------------------------------------------------|
pf_0 = c3d_f["data"]["platform"][0]  # Select the first platform
pf_1 = c3d_f['data']['platform'][1]  # Select the second platform
pf_2 = c3d_f['data']['platform'][2]  # Select the third platform
|--------------------------------------------------------------------|
pf_0['unit_force']          # Units of forces
pf_0['unit_moment']         # Units of moments
pf_0['unit_position']       # Units of center of pressure
|--------------------------------------------------------------------|
pf_0['cal_matrix']          # Calibration matrix
pf_0['corners']             # Position of the corners
pf_0['origin']              # Position of the origin
|--------------------------------------------------------------------|
pf_0['force']               # Force data
pf_0['moment']              # Moment data
pf_0['center_of_pressure']  # Center of pressure data
pf_0['Tz']                  # Moment at center of pressure data